In [12]:
import pandas as pd 
import numpy as np
import sklearn
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import PoissonRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler

post_data = pd.read_csv('/workspaces/Crowdfunding-Social-Media-Drivers/Data/Original_dataset/processed_data.csv')

targets = ['likes', 'shares', 'comments', 'positive_reactions', 'negative_reactions']
features = [x for x in post_data.columns if x not in targets]


features = post_data[features]
targets = post_data[targets]



In [13]:
def pre_process(x):
    scalable_features = ['likes_at_posting', 'followers_at_posting', 'post_views']
    noticable_entities = ['ORG', 'PERSON', 'DATE',
                          'CARDINAL', 'GPE', 'PRODUCT', 
                          'WORK_OF_ART','ORDINAL', 'MONEY',
                          'TIME', 'NORP']
    scaler = RobustScaler()
    x.loc[:,scalable_features] = scaler.fit_transform(x.loc[:,scalable_features])

    x = pd.concat([x, pd.get_dummies(x.type, prefix='type', drop_first=True).astype(int)], axis=1).drop('type', axis=1)
    x = pd.concat([x, pd.get_dummies(x.page_name, prefix='page_name').astype(int)], axis=1).drop('page_name', axis=1)
    for entity in noticable_entities:
        x[f'entity_{entity}'] = x.entities_identified.fillna('None').str.split(',').apply(lambda entity_list: entity in entity_list).astype(int)
    
    x = x.drop('entities_identified', axis=1)
    return x


def process_targets(y):
    y = y.copy()
    y = np.log(y+1)
    return y

def decode_targets(y):
    y = y.copy()
    y = np.exp(y)-1
    return y

In [14]:
processed_features = pre_process(features)
processed_targets = process_targets(targets)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(processed_features, processed_targets, test_size=0.2, random_state=42)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

y_test = decode_targets(y_test)

In [17]:
#xgb training
xgb_param_grid = {
    'n_estimators': [500, 1000, 2500, 5000],
    'learning_rate': [0.05, 0.001, 0.01],
    'max_depth': [3, 5, 10],
    'random_state': [42],
    'n_jobs': [-1]
}


xgb_grid_search = GridSearchCV(xgb.XGBRegressor(), xgb_param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
xgb_grid_search.fit(x_train, y_train)

xgb_best_params = xgb_grid_search.best_params_
xgb_best_estimator = xgb_grid_search.best_estimator_



print(xgb_best_params)

xgb_best_estimator.fit(x_train, y_train)

xgb_pred = xgb_best_estimator.predict(x_test)
xgb_pred = decode_targets(xgb_pred).astype(int)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_r2 = r2_score(y_test, xgb_pred)
# xgb_roc_auc = roc_auc_score(y_test, xgb_pred)

print(f'XGB RMSE: {xgb_rmse}')
print(f'XGB R2: {xgb_r2}')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.4s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.5s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.4s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.4s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   2.0s
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 500, 'n_jobs': -1, 'random_state': 42}
XGB RMSE: 29.70453285106309
XGB R2: 0.48085040544304947


In [ ]:
estimators = [100,150,200,250,500,750, 1000]
features = [0.33,0.5, 1, 'sqrt']
samples_leaf = [1,2,5,10]
bootstrap = [True, False]
min_samples_split = [2, 5, 10]


random_grid = {'estimator__n_estimators': estimators,
               'estimator__max_features': features,
               'estimator__min_samples_split': min_samples_split,
               'estimator__min_samples_leaf': samples_leaf,
               'estimator__bootstrap': bootstrap}

rf_grid_search = GridSearchCV(RandomForestRegressor(criterion="poisson"), random_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
rf_grid_search.fit(x_train, y_train)

rf_best_params = rf_grid_search.best_params_
rf_best_estimator = rf_grid_search.best_estimator_

print(rf_best_params)

rf_best_estimator.fit(x_train, y_train)
rf_pred = rf_best_estimator.predict(x_test)
rf_pred = decode_targets(rf_pred).astype(int)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_r2 = r2_score(y_test, rf_pred)

print(f'RF RMSE: {rf_rmse}')
print(f'RF R2: {rf_r2}')